In [1]:
import httpx
from openai import OpenAI
import os

client = OpenAI(
    http_client=httpx.Client(proxy="<YOUR PROXY>"),  
    api_key="<YOUR_KEY>".encode('ascii', 'ignore').decode('ascii')
    )

In [3]:
import torch, torchvision
from torch import nn
from torchvision import models, transforms, utils
from torch.utils.data import DataLoader
import torch, os
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import os

In [4]:
from tqdm import tqdm
import argparse
import json
import os
import pathlib
import re
from tqdm import tqdm
import time

NUM_SECONDS_TO_SLEEP = 0.5


import openai
from openai import OpenAI

In [5]:
import os
import glob
from pathlib import Path
import PIL
from PIL import Image

In [6]:
import base64
import requests
from openai import OpenAI


import matplotlib.colors as mcolors
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [7]:
with open('<NAME>.json', 'r') as file: #original bboxes coordinates
    orig_maira = json.load(file)

with open('<NAME>.json', 'r') as file: #bboxes coordinates produced with MedRPG on step 1
    medrpg = json.load(file)
    
with open('<NAME>.json', 'r') as file: #bboxes coordinates produced with Maira on step 2
    maira = json.load(file)

with open('<NAME>.json', 'r') as file:  #iou values for every picture produced with MedRPG on step 1
    medrpg_iou = json.load(file)
    
with open('<NAME>.json', 'r') as file: #iou values for every picture produced with Maira on step 1
    maira_iou = json.load(file)

In [11]:
gold_500 = pd.read_csv('gold_500.csv')
body_parts = gold_500.bbox_name.unique()

In [15]:

outliers = {}
n_tries = 5
iou_gpt = {}
workers = {}


for part in body_parts:

    workers[part] =[]
    iou_gpt[part] = []
    
    l = list(maira_iou[part].keys())
    for image in tqdm(l):
        
        path = f"with_gt_new/{part}/{image}"  #link to the folder with images
        if (os.path.exists(path)):
            img_try = Image.open(path).convert("RGB").resize((224, 224 ))

            # blue -original
            #red-maira
            #yellow - medrpg

    
            promt2 = f"You are given the image with three bounding boxes. The first bbox is red, the second bbox is blue, and the third is yellow. Please, analize the image and and tell what bbox describes the position of {part} better. Print Maira in case of the red bbox, MedRPG in case of the yellow bbox and Orig in case of the blue bbox.  Please, send ONLY one word"
 

            base64_image_1_224 = encode_image(path)

            response = client.chat.completions.create(
                    model="gpt-4o",
                    messages=[
                    {
                      "role": "user",
                      "content": [
                        {"type": "text", "text": promt2},
                        {
                          "type": "image_url",
                          "image_url": {
                              "url": f"data:image/jpeg;base64,{base64_image_1_224}",
                          },
                        },
                      ],
                    }
                  ],
                  max_tokens=300,
                  )

            response = response.choices[0].message.content



            maira_gt_iou = maira_iou[part][image]
            medrpg_gt_iou = medrpg_iou[part][image] 
            
            
            # in case chat gpt doesnot provide an answer accoring to the pattern we need to ask chat gpt again
            if response.lower() not in ["maira", "medrpg", "orig"]:
                for attempt in range(n_tries):
                    r = client.chat.completions.create(
                    model="gpt-4o",
                    messages=[
                    {
                      "role": "user",
                      "content": [
                        {"type": "text", "text": promt2},
                        {
                          "type": "image_url",
                          "image_url": {
                                  "url": f"data:image/jpeg;base64,{base64_image_1_224}",
                              },
                            },
                          ],
                        }
                      ],
                      max_tokens=300,
                      )
                    r = r.choices[0].message.content
                    if r.lower() in ["maira", "medrpg", "orig"]:
                        response = r
                        maira_gt_iou = maira_iou[part][image]
                        medrpg_gt_iou = medrpg_iou[part][image]
                        print("coorected response: ", r)
                        break
                    else:
                        if part not in outliers.keys():
                            #in case chatgpt still did not provide an answer we mark the picture as outlier
                            outliers[part] = [path]
                        else:
                            outliers[part].append(path)
                        maira_gt_iou = 0
                        medrpg_gt_iou = 0
                        print("incorrect response: ",r)

                        
                        
            if response.lower() == "maira":
                    iou_gpt[part].append(maira_gt_iou)
                    
            else:
                if response.lower() == "medrpg":
                    iou_gpt[part].append(medrpg_gt_iou) 
                   
                else:     
                    if response.lower() == "orig":
                        iou_gpt[part].append(1) 
                        
                    else:
                        iou_gpt[part].append(0) 
                        
            workers[part].append(response)
            

            with open('iou_with_gpt_v232323.json', 'w') as json_file:
                json.dump(iou_gpt, json_file, allow_nan=True)

            with open('workers_with_gpt_v232323.json', 'w') as json_file:
                json.dump(workers, json_file, allow_nan=True)
            with open('outliers_without_GT_232323.json', 'w') as json_file:
                json.dump(outliers, json_file, allow_nan=True)

        else:
            pass
    

100%|██████████| 8/8 [00:17<00:00,  2.19s/it]
0it [00:00, ?it/s]
